In [2]:
import pandas as pd 
import numpy as np
from bertopic import BERTopic 
import nltk 
from nltk.corpus import stopwords 
import re
import string

#from nltk.tokenize import word_tokenize
#from nltk.stem import WordNetLemmatizer

In [19]:
#switched to indeed for better interpretation 
df = pd.read_csv("/Users/hanna/Scape-Save-DAEN690/indeed.csv.gz")
df = df[0:300]

C:\Users\hanna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\IPython\core\interactiveshell.py:3369: DtypeWarning:

Columns (15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50) have mixed types.Specify dtype option on import or set low_memory=False.



In [20]:
stop_words = set(stopwords.words('english'))
df['description_text'] = df['description_text'].apply(lambda x: str(x).lower())
df['description_text'] = df['description_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))
df['description_text'] = df['description_text'].apply(lambda x: ''.join([word for word in x if word.isalpha() or word.isspace()]))
df['description_text'] = df['description_text'].apply(lambda x: x.strip())
#remove punctuation
df['description_text'] = df['description_text'].apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), '' , x))
#remove words and digits
df['description_text'] = df['description_text'].apply(lambda x: re.sub('W*dw*','',x))

In [22]:
#using only the base BERTopic model, without any additionals functions added to enhance the model 
topic_model = BERTopic(language="english", calculate_probabilities=True, verbose=True)
topics, probs = topic_model.fit_transform(df['description_text'])

Batches: 100%|██████████| 10/10 [00:34<00:00,  3.49s/it]
2023-03-12 12:55:07,005 - BERTopic - Transformed documents to Embeddings
2023-03-12 12:55:09,997 - BERTopic - Reduced dimensionality
2023-03-12 12:55:10,037 - BERTopic - Clustered reduced embeddings


In [23]:
freq = topic_model.get_topic_info(); freq.head(5)
topic_model.get_topic(0)  # Select the most frequent topic

[('security', 0.07041576324841314),
 ('experience', 0.04040577300234382),
 ('information', 0.03979439768303841),
 ('systems', 0.032329151226696044),
 ('management', 0.023382431940169077),
 ('support', 0.023280529833261135),
 ('system', 0.022924571341824258),
 ('cybersecurity', 0.022137335612215557),
 ('work', 0.02197178961115876),
 ('network', 0.021073318355581967)]

In [24]:
topic_model.visualize_barchart(top_n_topics=5)

In [34]:
freq

,Topic,Count,Name
0,-1,121,-1_experience_security_work_team
1,0,101,0_security_experience_information_systems
2,1,48,1_experience_business_work_team
3,2,30,2_security_experience_work_information


In [39]:
from umap import UMAP
from hdbscan import HDBSCAN
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer

from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired
from bertopic.vectorizers import ClassTfidfTransformer


# Step 1 - Extract embeddings
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Step 2 - Reduce dimensionality
umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine')

# Step 3 - Cluster reduced embeddings
hdbscan_model = HDBSCAN(min_cluster_size=15, metric='euclidean', cluster_selection_method='eom', prediction_data=True)

# Step 4 - Tokenize topics
vectorizer_model = CountVectorizer(stop_words="english")

# Step 5 - Create topic representation
ctfidf_model = ClassTfidfTransformer()

# Step 6 - (Optional) Fine-tune topic representations with 
# a `bertopic.representation` model
representation_model = KeyBERTInspired()

# All steps together
topic_model = BERTopic(
  embedding_model=embedding_model,          # Step 1 - Extract embeddings
  umap_model=umap_model,                    # Step 2 - Reduce dimensionality
  hdbscan_model=hdbscan_model,              # Step 3 - Cluster reduced embeddings
  vectorizer_model=vectorizer_model,        # Step 4 - Tokenize topics
  ctfidf_model=ctfidf_model,                # Step 5 - Extract topic words
  representation_model=representation_model) # Step 6 - (Optional) Fine-tune topic represenations


In [40]:
topics, probs = topic_model.fit_transform(df['description_text'])

2023-03-12 21:46:06,091 - BERTopic - Transformed documents to Embeddings
2023-03-12 21:46:10,772 - BERTopic - Reduced dimensionality
2023-03-12 21:46:10,828 - BERTopic - Clustered reduced embeddings


In [43]:
#refined extracted topics 
topic_model.visualize_barchart(top_n_topics=5)

In [44]:
freq = topic_model.get_topic_info(); freq.head(5)
topic_model.get_topic(0)  # Select the most frequent topic

[('cybersecurity', 0.46039912),
 ('security', 0.42549816),
 ('compliance', 0.36047423),
 ('requirements', 0.3311742),
 ('certification', 0.32263905),
 ('services', 0.31602678),
 ('secure', 0.31525803),
 ('management', 0.3089983),
 ('infrastructure', 0.30520394),
 ('applications', 0.2977597)]

In [82]:
freq #more variety of topics compared to the first model 

,Topic,Count,Name
0,-1,133,-1_cybersecurity_security_services_management
1,0,98,0_cybersecurity_security_compliance_requirements
2,1,45,1_consulting_enterprise_opportunities_services
3,2,24,2_enterprise_services_cybersecurity_organization


In [79]:
#reduce outliers and update topics before fitting the updated model 
#source: https://maartengr.github.io/BERTopic/getting_started/outlier_reduction/outlier_reduction.html#exploration
new_topic= topic_model.reduce_outliers(df['description_text'], topics)
topic_model.update_topics(df['description_text'], topics=new_topic)

In [ ]:
sentence_model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = sentence_model.encode(df['description_text'], show_progress_bar=True)
reduced_embeddings = UMAP(n_neighbors=10, n_components=2, 
                          min_dist=0.0, metric='cosine').fit_transform(embeddings)

topic_model.visualize_documents(df['description_text'], reduced_embeddings=reduced_embeddings, 
                                hide_document_hover=True, hide_annotations=True)

In [84]:
hierarchical_topics = topic_model.hierarchical_topics(df['description_text'])
topic_model.visualize_hierarchy(hierarchical_topics=hierarchical_topics)

IndexError: index 2 is out of bounds for axis 0 with size 2